In [1]:
from typing import Optional
import numpy as np
import plotly.graph_objects as go

## Homogeneous coordinates

In this context, we use $[u\ v\ 1]^{T}$ to represent a 2D point position in pixel coordinates and 
$[x_{w}\ y_{w}\ z_{w}\ 1]^{T}$ is used to represent a 3D point position in world coordinates. In both cases, they are represented in homogeneous coordinates (i.e. they have an additional last component, which is initially, by convention, a 1), which is the most common notation in robotics and rigid body transforms.

## World to Camera Projection

Referring to the pinhole camera model, a camera matrix $M$ is used to denote a projective mapping from world coordinates to pixel coordinates.

$$
{\begin{bmatrix}wu\\wv\\w\end{bmatrix}}=K\,{\begin{bmatrix}R&T\end{bmatrix}}{\begin{bmatrix}x_{w}\\y_{w}\\z_{w}\\1\end{bmatrix}}=M{\begin{bmatrix}x_{w}\\y_{w}\\z_{w}\\1\end{bmatrix}}
$$

## Intrinsic parameters

$$
K={\begin{bmatrix}\alpha _{x}&\gamma &u_{0}\\0&\alpha _{y}&v_{0}\\0&0&1\end{bmatrix}}
$$

The parameters $\alpha _{x}=f\cdot m_{x}$ and $\alpha _{y}=f\cdot m_{y}$ represent focal length in terms of pixels, where 
$m_{x}$ and $m_{y}$ are the inverses of the width and height of a pixel on the projection plane. 

$f$ is the focal length in terms of distance. The focal length is the distance between the pinhole and the film (a.k.a. image plane).


$\gamma$ represents the skew coefficient between the x and the y axis, and is often 0. $u_{0}$ and $v_{0}$ represent the principal point, which would be ideally in the center of the image.

In [2]:
def compute_intrinsic_matrix(f, w, h, u_0, v_0, gamma=0.0, res_w=100, res_h=100):
    return np.array([
        [f, gamma, u_0],
        [0.0, f, v_0],
        [0.0, 0.0, 1.0]
    ])

In [3]:
def angles2Matrix(alpha, beta, gamma):
    """
    Convert Euler angles to rotation matrix.
    """
    alpha = np.radians(alpha)
    beta = np.radians(beta)
    gamma = np.radians(gamma)
    Rz = np.array([
        [np.cos(alpha), -np.sin(alpha), 0],
        [np.sin(alpha), np.cos(alpha), 0],
        [0, 0, 1]
    ])
    Ry = np.array([
        [np.cos(beta), 0, np.sin(beta)],
        [0, 1, 0],
        [-np.sin(beta), 0, np.cos(beta)]
    ])
    Rx = np.array([
        [1, 0, 0],
        [0, np.cos(gamma), -np.sin(gamma)],
        [0, np.sin(gamma), np.cos(gamma)]
    ])
    return Rz @ Ry @ Rx

def plot_world_coordinates(fig: go.Figure):
    # Add the x-axis
    fig.add_trace(go.Scatter3d(
        x=[0, 2],
        y=[0, 0],
        z=[0, 0],
        mode='lines',
        name='x-axis'
    ))

    # Add the y-axis
    fig.add_trace(go.Scatter3d(
        x=[0, 0],
        y=[0, 2],
        z=[0, 0],
        mode='lines',
        name='y-axis'
    ))

    # Add the z-axis
    fig.add_trace(go.Scatter3d(
        x=[0, 0],
        y=[0, 0],
        z=[0, 2],
        mode='lines',
        name='z-axis'
    ))

def to_homogeneous(points):
    pad = np.ones((points.shape[:-1]+(1,)), dtype=points.dtype)
    return np.concatenate([points, pad], axis=-1)

In [24]:
import dash
from dash import dcc, html
from dash.dependencies import Input, Output

def plot_camera(
        R: np.ndarray,  # R camera coordinate size (3,3)
        t: np.ndarray,  # t camera coordinate size (3,)
        K: np.ndarray,  # size (3,3)
        screen_w, screen_h,
        far_dist=3.0,
        color: str = 'rgb(0, 0, 255)',
        size: float = 1.0):
    """Plot a camera frustum from pose and intrinsic matrix."""
    W_c, H_c = screen_w, screen_h

    f = K[0, 0]
    u_0, v_0 = K[0, 2], K[1, 2]
    screen_dist = 1 / (1/f - 1/far_dist)

    s = np.array([[0, 0], [W_c, 0], [W_c, H_c], [0, H_c], [0, 0]], dtype=np.float32)
    screen_corners = to_homogeneous(s)
    screen_corners[:, :2] -= np.array([W_c, H_c], dtype=np.float32) / 2
    screen_corners[:, 2] = screen_dist

    far_plane_corners = to_homogeneous(s) 
    far_plane_corners[:, :2] -= np.array([W_c, H_c], dtype=np.float32) / 2
    far_plane_corners[:, :2] *=  far_dist / screen_dist
    far_plane_corners[:, 2] = far_dist

    screen_corners = (screen_corners) @ R.T + t
    far_plane_corners = (far_plane_corners) @ R.T + t

    x, y, z = np.concatenate(([t], screen_corners, far_plane_corners)).T
    i = [0, 0, 0, 0,    0, 0, 0, 0]
    j = [1, 2, 3, 4,    6, 7, 8, 9]
    k = [2, 3, 4, 1,    7, 8, 9, 6]

    triangles = np.vstack((i, j, k)).T
    vertices = np.concatenate(([t], screen_corners, far_plane_corners))
    tri_points = np.array([
        vertices[i] for i in triangles.reshape(-1)
    ])
    x, y, z = tri_points.T
    return x, y, z

fig = go.Figure()
fig.update_layout(
    title='Camera',
    autosize=False,
    width=600,
    height=600,
    scene=dict(
        xaxis=dict(range=[-5, 5]),
        yaxis=dict(range=[-5, 5]),
        zaxis=dict(range=[-5, 5]),
        aspectratio=dict(x=1, y=1, z=1),
    ),
)

screen_w, screen_h = 2, 2
f = 1.0
u_0 = 1.0
v_0 = 1.0
K = compute_intrinsic_matrix(f, screen_w, screen_h, u_0, v_0)

R = np.eye(3)
t = np.array([0.0,
              0.0,
              0.0])
R_wc = R.T
t_wc = -R.T @ t

x, y, z = plot_camera(R_wc, t_wc, K, f, screen_w, screen_h)

pyramid = go.Scatter3d(
    x=x, y=y, z=z, mode='lines', line=dict(color="rgb(0, 0, 255)", width=1), showlegend=False,
    hovertemplate="demo-text".replace('\n', '<br>'))
fig.add_trace(pyramid)

plot_world_coordinates(fig)


app = dash.Dash(__name__)
app.layout = html.Div([
    html.Div(children=[
        html.Div(children=[
            dcc.Graph(id='camera-plot', figure=fig),
        ], style={'display': 'inline-block'}),
    ], style={ 'display': 'inline-block'}),
    html.Div(children=[
        html.Div('mean-x', style={'color': 'white', 'fontSize': 14, "padding-left": "20px"}),
        dcc.Slider(id='mean-x', min=-4, max=4, step=0.1, value=0, marks={i: str(i) for i in range(-10, 11)}, updatemode='drag'),
        html.Div('mean-y', style={'color': 'white', 'fontSize': 14, "padding-left": "20px"}),
        dcc.Slider(id='mean-y', min=-4, max=4, step=0.1, value=0, marks={i: str(i) for i in range(-10, 11)}, updatemode='drag'),
        html.Div('mean-z', style={'color': 'white', 'fontSize': 14, "padding-left": "20px"}),
        dcc.Slider(id='mean-z', min=-4, max=4, step=0.1, value=0, marks={i: str(i) for i in range(-10, 11)}, updatemode='drag'),
        html.Div('alpha', style={'color': 'white', 'fontSize': 14, "padding-left": "20px"}),
        dcc.Slider(id='alpha', min=0, max=360, step=1, value=0, marks={i: str(i) for i in range(0, 361, 30)}, updatemode='drag'),
        html.Div('beta', style={'color': 'white', 'fontSize': 14, "padding-left": "20px"}),
        dcc.Slider(id='beta', min=0, max=360, step=1, value=0, marks={i: str(i) for i in range(0, 361, 30)}, updatemode='drag'),
        html.Div('gamma', style={'color': 'white', 'fontSize': 14, "padding-left": "20px"}),
        dcc.Slider(id='gamma', min=0, max=360, step=1, value=0, marks={i: str(i) for i in range(0, 361, 30)}, updatemode='drag'),
        html.Div('focal', style={'color': 'white', 'fontSize': 14, "padding-left": "20px"}),
        # dcc.Slider(id='focal', min=2, max=10, step=0.1, value=1, marks={i: str(i) for i in range(2, 10)}, updatemode='drag'),
        dcc.Slider(id='focal', min=0.1, max=1.5, step=0.1, value=1, updatemode='drag'),
        html.Div('p-x', style={'color': 'white', 'fontSize': 14, "padding-left": "20px"}),
        dcc.Slider(id='p-x', min=0, max=1, step=0.1, value=1, marks={i: str(i) for i in range(0, 2)}, updatemode='drag'),
        html.Div('p-y', style={'color': 'white', 'fontSize': 14, "padding-left": "20px"}),
        dcc.Slider(id='p-y', min=0, max=1, step=0.1, value=1, marks={i: str(i) for i in range(0, 2)}, updatemode='drag'),
    ], style={'width': '30%', 'display': 'inline-block'}),
])


@app.callback(
    Output('camera-plot', 'extendData'),
    [Input('mean-x', 'value'),
     Input('mean-y', 'value'),
     Input('mean-z', 'value'),
     Input('alpha', 'value'),
     Input('beta', 'value'),
     Input('gamma', 'value'),
     Input('focal', 'value'),
     Input('p-x', 'value'),
     Input('p-y', 'value')]
)
def update_camera(*data):
    mean_x, mean_y, mean_z, alpha, beta, gamma, f, u_0, v_0 = data

    R = angles2Matrix(alpha, beta, gamma)
    t = np.array([mean_x, mean_y, mean_z])

    K = compute_intrinsic_matrix(f, screen_w, screen_h, u_0, v_0)

    x, y, z = plot_camera(R, t, K, screen_w, screen_h)
    return [{
        'x': [x],
        'y': [y],
        'z': [z]
    }, [0], 24]


app.run_server(debug=True, port=9999)
